In [ ]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu
from skimage.morphology import closing, square

In [ ]:
video_path = "../video/rilevamento-intrusioni-video.wm"
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Not opened!"

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
length = total_frame_count / fps

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[I] Video FPS: {fps}")
print(f"[I] Video Total frame count: {total_frame_count}")
print(f"[I] Video Length: {length}")
print(f"[I] Video Frame Width: {width}")
print(f"[I] Video Frame height: {height}")

In [ ]:
# Trying to understand MOG2 parameters

# # BackgroundRatio --> ok default (0.9) per ora quindi
# If a foreground pixel keeps semi-constant value for about backgroundRatio*history frames, it's considered background and added to the model 
# as a center of a new component.
# "In altre parole, TB controlla quanto velocemente il modello di sfondo si adatta ai cambiamenti nella scena."
# - Un valore più alto rende il modello di sfondo più adattabile e meno sensibile ai movimenti.
# - Un valore più basso rende il modello di sfondo più stabile e più sensibile ai movimenti, ma anche più lento ad adattarsi.

# valore simile a quelo di default --> lasciamo quello di default
# print("Mean Background ratio: ", np.mean(ratios))

# # VarThreshold

In [ ]:
# """
#     Performed a sorted insertion of pv into the frame buffer in the pi row.
#     pv: pixel value
#     pi: pixel index
#     frame_buffer (global)
# """
# def sortAndInsert(pv, pi, ):
#     global frame_buffer
#     insertion_idx = np.searchsorted(frame_buffer[pi], pv)
#     if insertion_idx == len(frame_buffer[pi]):
#         insertion_idx =- 1

#     frame_buffer[pi, insertion_idx] = pv

#cv2.imshow("frame_copy", frame_contours)
# largestContour = max(contours, key = cv2.contourArea) # get largest contour
# rect = cv2.minAreaRect(largestContour)
# box = np.int64(cv2.boxPoints(rect))
# cv2.drawContours(frame_contours, [box], 0, (0,0,255), 1)
# cv2.imshow("frame_contours", frame_contours)

# cv2.imshow(f"temporalMedianBackground-frame-{frame_count}", temporalMedianBackground)
# cmd = cv2.waitKey(0)
# cv2.destroyWindow(f"temporalMedianBackground-frame-{frame_count}")
# if cmd == ord("q"):
#     break
# if cmd == ord("n"):
#     continue

# alpha = 3.0 # Contrast control
# beta = 25 # Brightness control
# # call convertScaleAbs function
# #adjusted = cv2.convertScaleAbs(mog2_fgmask, alpha=alpha, beta=beta)


# plt.hist(frame_diff.ravel(), 256, [0, 256])
# plt.show()


            
# Disegna la bounding box
#cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)

#cv2.imshow("frame_contours", frame_contours)


# analysis = cv2.connectedComponentsWithStats(tmb_fgmask, 8, cv2.CV_32S)
# (totalLabels, label_ids, values, centroid) = analysis 

# # Initialize a new image to store  
# # all the output components 
# output = np.zeros(tmb_fgmask.shape, dtype="uint8") 

# # Loop through each component 
# for i in range(1, totalLabels): 
    
#     # Area of the component 
#     area = values[i, cv2.CC_STAT_AREA]  
    
#     if (area > 140) and (area < 400): 
#         componentMask = (label_ids == i).astype("uint8") * 255
#         output = cv2.bitwise_or(output, componentMask) 

# cv2.imshow("Filtered Components", output)
# 
# 
#mog2 = cv2.createBackgroundSubtractorMOG2(history=fps*2, varThreshold=20, detectShadows=False)

# print(mog2.getHistory())
# print(mog2.getVarThreshold())
# print(mog2.getBackgroundRatio())
# print(mog2.getNMixtures())
# print(mog2.getVarThresholdGen())
# print(mog2.getVarMin())
# print(mog2.getVarMax())
# print(mog2.getComplexityReductionThreshold())
# print(mog2.getVarInit())

#mog2.setComplexityReductionThreshold(0)
#mog2.setBackgroundRatio(0.7)
#mog2.setNMixtures(1)
#mog2.setVarThresholdGen(30) 

#temporalMedianBackground_tmp = np.median(frames_tmp, axis=0).astype(dtype=np.uint8)
#cv2.imshow("temporal background - true one", temporalMedianBackground)
# make addWeighted between temporalMedianBackground and frame just where the background_mask is 1
#masked_img = np.where(background_mask[..., None] == 255, frame[..., None], 0) # dove c'è il background mask metti frame, altrimenti 0
#cv2.imshow("masked_img", masked_img)
#temporalMedianBackground = cv2.addWeighted(temporalMedianBackground, 0.7, masked_img, 0.3, 0)
#temporalMedianBackground = cv2.bitwise_and(temporalMedianBackground, temporalMedianBackground, background_mask)



In [ ]:
def preprocessing(frame, type):
    """
        Apply all the preprocessing steps to a copy of the passed frame.

        Arguments:
            frame (MatLike): frame to process
            type (ProcessingType): which type of preprocess

        Returns:
            (MatLike): a processed copy of the frame
    """
    
    assert isinstance(type, ProcessingType), "type must be an instance of ProcessingType (Enum)"
    output = frame.copy()
    
    if type == ProcessingType.DENOISE:
        # output = cv2.medianBlur(output, 5)
        output = cv2.GaussianBlur(output, (5, 5), 2)
        
    elif type == ProcessingType.EDGES:
        # Laplacian sharpening
        # laplacian = cv2.Laplacian(output)
        # laplacian = cv2.convertScaleAbs(output)
        # output = cv2.add(output, -laplacian)
    
        sharpening = np.array([ [0 , -1,  0],
                                [-1,  5, -1],
                                [0 , -1,  0]])
        output = cv2.filter2D(output, -1, sharpening)

    else:
        raise ValueError("ProcessingType.type not found")
    
    return output

In [ ]:
def compute_learning_rate(frame_count, history_length, dynamic = True, bias = 0):
    """
        Computes the learning rate

        Arguments:
            frame_count (int) : current frame number
            history_lenght (int): number of frames in buffer
            dynamic (boolean)
            bias (float): constant added to the computed learning rate

        Returns:
            float: the learning rate to use
        
    """
    
    assert frame_count > 0, "Frame Count must be greater than zero"
    if dynamic and frame_count < history_length:
        return (1 / frame_count) + bias
    
    return (1 / history_length) + bias

In [ ]:
from skimage.segmentation import flood_fill
def postprocessing(fgmask, type):
    """
        Apply a pipeline of morphological operators to improve the segmentation
    """
    assert isinstance(type, ProcessingType), "type must be an instance of ProcessingType (Enum)"
    # cv2.imshow("fgmask UNPROCESSED", fgmask)
    output = fgmask.copy()
    element = np.ones((3,3), np.uint8)

    if type == ProcessingType.MOG:
        output = cv2.medianBlur(output, 3)
        # output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, element, iterations=1)
        
        # output = flood_fill(output, (1,1), 127, connectivity=25)
    else:
        raise ValueError("ProcessingType.type not found")
            
    return output

In [ ]:
# from scipy import ndimage as ndi
# from skimage import morphology
# def apply_hysteresis_threshold(image, low, high):
#     """Apply hysteresis thresholding to ``image``.

#     This algorithm finds regions where ``image`` is greater than ``high``
#     OR ``image`` is greater than ``low`` *and* that region is connected to
#     a region greater than ``high``.

#     Parameters
#     ----------
#     image : (M[, ...]) ndarray
#         Grayscale input image.
#     low : float, or array of same shape as ``image``
#         Lower threshold.
#     high : float, or array of same shape as ``image``
#         Higher threshold.

#     Returns
#     -------
#     thresholded : (M[, ...]) array of bool
#         Array in which ``True`` indicates the locations where ``image``
#         was above the hysteresis threshold.

#     Examples
#     --------
#     >>> image = np.array([1, 2, 3, 2, 1, 2, 1, 3, 2])
#     >>> apply_hysteresis_threshold(image, 1.5, 2.5).astype(int)
#     array([0, 1, 1, 1, 0, 0, 0, 1, 1])

#     References
#     ----------
#     .. [1] J. Canny. A computational approach to edge detection.
#            IEEE Transactions on Pattern Analysis and Machine Intelligence.
#            1986; vol. 8, pp.679-698.
#            :DOI:`10.1109/TPAMI.1986.4767851`
#     """
#     low = np.clip(low, a_min=None, a_max=high)  # ensure low always below high
#     mask_low = image > low
#     mask_high = image > high

#     # remove small objects
#     processed = morphology.remove_small_objects(mask_high.astype(bool), min_size=25, connectivity=5).astype(int)

#     # black out pixels
#     mask_x, mask_y = np.where(processed == 0)
#     mask_high[mask_x, mask_y] = 0

#     # Connected components of mask_low
#     labels_low, num_labels = ndi.label(mask_low)
#     # Check which connected components contain pixels from mask_high
#     sums = ndi.sum(mask_high, labels_low, np.arange(num_labels + 1))
#     connected_to_high = sums > 0
#     thresholded = connected_to_high[labels_low]
#     return thresholded

In [ ]:
from skimage import morphology, img_as_ubyte, filters
def background_subtraction(frame, background, adaptive, args):
    """
        Computes the absolute difference between the frame and the background then applies a threshold to segment the
        background and the foreground.

        Arguments:
            frame (Mat): image, current frame
            background (Mat): image, estimated reference frame
            args:
             - threshold (Int)=25: if defined, is the value of the static threshold
             - C (float)=None: if defined, it will use and adaptive thresholding method and it represent the value of C (Constant subtracted from the mean or weighted mean)
            
        Returns:
            fgmask: binary image with the foreground white (255).
    """
    diff = cv2.absdiff(frame, background)
    # diff_edges = diff_canny_edges(frame, background)
    
    # cv2.imshow("diff", diff)
    # cv2.imshow("diff_edges", diff_edges)

    if adaptive:
        (C, block_size) = args
        fgmask = cv2.adaptiveThreshold(diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, C)
    else:
        (t_low, t_high) = args
        hysteresis = filters.apply_hysteresis_threshold(diff, t_low, t_high)
        # TODO: remove edge to create a black contour
        fgmask = img_as_ubyte(hysteresis)

        # fgmask = combine_fgmask_with_edges(fgmask, diff_edges)

    return fgmask
        

In [ ]:
# def get_roi_foreground(fgmask):
#     enclosed_mask = morphology.convex_hull_image(fgmask)
#     enclosed_mask = img_as_ubyte(enclosed_mask)
#     # cv2.imshow("enclosed_mask", enclosed_mask)

#     enclosed_mask_copy = fgmask.copy()
#     enclosed_mask_copy = cv2.cvtColor(enclosed_mask_copy, cv2.COLOR_GRAY2BGR)
#     contours, _ = cv2.findContours(enclosed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     if len(contours) > 0:
#         largestContour = max(contours, key = cv2.contourArea) # get largest contour
#         rect = cv2.minAreaRect(largestContour)
#         box  = np.int64(cv2.boxPoints(rect))
#         cv2.drawContours(enclosed_mask_copy, [box], 0, (0,0,255), 4)
#         cv2.imshow("enclosed_mask contours", enclosed_mask_copy)
#     return enclosed_mask

In [ ]:
def get_roi_foreground(fgmask):
    blur = cv2.GaussianBlur(fgmask, (5,5), 3)
    thresh = threshold_otsu(blur)
    if thresh == 0:
        return False, None
    
    bw = closing(blur >= thresh, square(7))
    #erode = cv2.erode(bw.astype(np.uint8) * 255, None, iterations=3)
    #cv2.imshow("erode", erode)
    erode = cv2.medianBlur(bw.astype(np.uint8) * 255, 3)
    dilated = cv2.dilate(erode, None, iterations=5)
    #cv2.imshow("dilate", dilated)        

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    real_countours = []
    for contour in contours:
        # (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) > 100:
            real_countours.append(contour)
        
    
    if len(real_countours) == 0:
        return False, None
    
    # Define the ROI 
    x_min = min([np.min(cnt[:, :, 0]) for cnt in real_countours])
    x_max = max([np.max(cnt[:, :, 0]) for cnt in real_countours])
    y_min = min([np.min(cnt[:, :, 1]) for cnt in real_countours])
    y_max = max([np.max(cnt[:, :, 1]) for cnt in real_countours])

    padding = 10
    x_min = max(x_min - padding, 0)
    x_max = min(x_max + padding, 320)
    y_min = max(y_min - padding, 0)
    y_max = min(y_max + padding, 240)

    return True, (x_min, x_max, y_min, y_max)

In [ ]:
def exponential_operator(img, r):
    exp_img = ((img/255)**r) * 255
    return exp_img

In [ ]:
def diff_canny_edges(frame, background):
    """
        Computes the difference of the Canny's egdes of the current frame and Canny's egdes of the background.
    
        Arguments:
            frame (Mat): current frame
            background (Mat): estimated reference frame
            
        Returns:
            img: binary image with differences of edges
    """
    upthresh = 150
    lothresh = upthresh // 2

    background = preprocessing(background, ProcessingType.EDGES)
    edge_background = cv2.Canny(background, lothresh, upthresh)
    
    # enlarge background edges to better cancel out with the ones of the frame
    # elem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    # edge_background = cv2.dilate(edge_background, elem, iterations=1)
    
    frame = preprocessing(frame, ProcessingType.EDGES)
    edge_frame = cv2.Canny(frame, lothresh, upthresh)

    # edge_diff will contain the edges that are present in the frame but not in the background
    # frame  back      output
    #     0     0 -->       0
    #   255     0 -->     255
    #     0   255 -->       0      (0 - 255 = -255 --> 0)
    #   255   255 -->       0
    edge_diff = edge_frame - np.min([edge_frame, edge_background], axis=0)
    # cv2.imshow("edge_diff", edge_diff)

    processed = morphology.remove_small_objects(edge_diff.astype(bool), min_size=6, connectivity=5).astype(int)

    # black out pixels
    mask_x, mask_y = np.where(processed == 0)
    edge_diff[mask_x, mask_y] = 0
    
    return edge_diff



In [ ]:
# [ CONNECTED COMPONENT ANALYSIS ]

# loop over the number of unique connected component labels
# for i in range(0, numLabels):
#     # if this is the first component then we examine the
#     # *background* (typically we would just ignore this
#     # component in our loop)
#     if i == 0:
#         text = f"examining component {i+1}/{numLabels} (background)"
#     # otherwise, we are examining an actual connected component
#     else:
#         text = f"examining component {i+1}/{numLabels}"
#     # print a status message update for the current connected
#     # component
#     print("[INFO] {}".format(text))
#     # extract the connected component statistics and centroid for
#     # the current label
#     x = stats[i, cv2.CC_STAT_LEFT]
#     y = stats[i, cv2.CC_STAT_TOP]
#     w = stats[i, cv2.CC_STAT_WIDTH]
#     h = stats[i, cv2.CC_STAT_HEIGHT]
#     area = stats[i, cv2.CC_STAT_AREA]
#     (cX, cY) = centroids[i]

#     # clone our original image (so we can draw on it) and then draw
#     # a bounding box surrounding the connected component along with
#     # a circle corresponding to the centroid
#     output = image.copy()
#     cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 3)
#     cv2.circle(output, (int(cX), int(cY)), 4, (0, 0, 255), -1)    

#     # construct a mask for the current connected component by
#     # finding a pixels in the labels array that have the current
#     # connected component ID
#     componentMask = (labels == i).astype("uint8") * 255
#     # show our output image and connected component mask
#     cv2.imshow("Output", output)
#     cv2.imshow("Connected Component", componentMask)
#     cv2.waitKey(0)

In [ ]:
def imshow_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    # cv2.imshow('labeled.png', labeled_img)
    return labeled_img


In [ ]:
from enum import Enum

# class syntax
class ProcessingType(Enum):
    DENOISE = 1
    BACKGROUND_FRAME = 2
    EDGES = 3
    MOG = 4

In [ ]:
# def combine_fgmask_with_edges(fgmask, edges):
#     output = cv2.bitwise_or(fgmask, edges)
#     cv2.imshow("without Canny Edges", fgmask)
    
#     # Combine edges
#     output = cv2.bitwise_or(fgmask, edges)
#     cv2.imshow("with Canny Edges", output)

#     # Process image
#     kernel = np.ones((8,8) , np.uint8)
#     output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, kernel, iterations=2)
#     cv2.imshow("test", output)

#     # Remove the same added egdes
#     dilate_edges = cv2.dilate(edges, kernel, iterations=2)
#     dilate_edges = cv2.morphologyEx(dilate_edges, cv2.MORPH_CLOSE, kernel, iterations=2)
#     found, (x_min, x_max, y_min, y_max) = getRectangularROI(fgmask)
#     mask = np.zeros_like(output)
#     mask[x_min:x_max, y_min:y_max] = 1

#     output = cv2.bitwise_and(output, cv2.bitwise_not(dilate_edges), mask)
#     cv2.imshow("test 2", output)

#     # Close the small gaps created
#     kernel = np.ones((3,3) , np.uint8)
#     output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, kernel, iterations=1)

#     cv2.imshow("test 3", output)
#     return output


In [ ]:
# HYPERPARAMTERS
FRAME_BUFFERED_PER_SECOND = 2               # 2 images added each second to frame buffer
MAX_HISTORY = 3 * fps                       # 2x12 frames stored in the "circular buffer"
SKIP_FRAMES = fps // FRAME_BUFFERED_PER_SECOND

STATIC_THRESHOLD = 35
STATIC_THRESHOLD_HYSTERESIS = 25

LEARNING_PHASE = 5 * fps                    # Initialization phase for the background subtractor to estimate the reference frame

# MOG2 Background Subtractor
mog2 = cv2.createBackgroundSubtractorMOG2()
mog2.setHistory(LEARNING_PHASE)
mog2.setBackgroundRatio(0.70)

BLACK = 0
mog2.setDetectShadows(True)
mog2.setShadowValue(BLACK)                  # Detect shadows and hide them
mog2.setShadowThreshold(0.75)

# mog2.setVarInit(5)                          # default: 15
# mog2.setVarMax(25)                          # default: 5 * 15
# mog2.setVarMin(4)                           # default: 4

# The main threshold on the squared Mahalanobis distance to decide if the sample is 
# well described by the background model or not. Related to Cthr from the paper. 
# mog2.setVarThreshold(5**2)                  # default: 16

# Threshold for the squared Mahalanobis distance that helps decide when a sample is close to the existing 
# components (corresponds to Tg in the paper). If a pixel is not close to any component, it is considered
# foreground or added as a new component. 3 sigma => Tg=3*3=9 is default. A smaller Tg value generates 
# more components. A higher Tg value may result in a small number of components but they can grow too large. 
# mog2.setVarThresholdGen(4**2)               # default:  9

start = time.time()

frame_buffer = []
frame_count = 0
skip_count = 0

temporalMedianBackground = None
fgmask_diff = None

font = cv2.FONT_HERSHEY_SIMPLEX

kernel = np.ones((3,3),np.uint8) # combined mask

while(cap.isOpened()):
    ret, frame_original = cap.read()
    if not ret or frame_original is None:
        cap.release()
        print("Released Video Resource")
        break

    frame_count += 1
    skip_count += 1

    frame = cv2.cvtColor(frame_original, cv2.COLOR_BGR2GRAY)
    frame = preprocessing(frame, ProcessingType.DENOISE)

    # cv2.imshow("current frame", frame)

    # -------------------------------
    # |   TEMPORAL MEDIAN FILTER    |
    # -------------------------------
    # TODO: port np.ma.MaskedArray and np.ma.median to create a selective updated temporal median
    if len(frame_buffer) == 0 or skip_count == SKIP_FRAMES:
        skip_count = 0
        frame_buffer.append(frame)

        if len(frame_buffer) > MAX_HISTORY:
            frame_buffer.pop(0)

        temporalMedianBackground = np.median(frame_buffer, axis=0).astype(dtype=np.uint8)
        temporalMedianBackground_copy = temporalMedianBackground.copy()
        cv2.putText(temporalMedianBackground_copy, f"FRAME: {frame_count}/{total_frame_count}", (5, 25), font, 0.5, (0, 0, 0), 1) 
        cv2.imshow("temporalMedianBackground", temporalMedianBackground_copy)

    # -------------------------------
    # |            MOG2             |
    # -------------------------------
    learning_rate = compute_learning_rate(frame_count, mog2.getHistory(), dynamic=False)
    
    mog2_fgmask = mog2.apply(frame, learning_rate)
    mog2_fgmask = postprocessing(mog2_fgmask, ProcessingType.MOG)
    # cv2.imshow("mog2_fgmask", mog2_fgmask)
    
    # fgmask = combine_fgmask_with_edges(mog2_fgmask, edge_diff)
    
    if frame_count > LEARNING_PHASE:
        # [ GET A ROI OF THE MOG2 FOREGROUND MASK ]
        found, roi = get_roi_foreground(mog2_fgmask)
        
        args = (STATIC_THRESHOLD_HYSTERESIS, STATIC_THRESHOLD)
        tmb_fgmask = background_subtraction(frame, temporalMedianBackground, False, args)
        # cv2.imshow("tmb_fgmask", tmb_fgmask)
        
        # [ COMBINE FGMSKS IN ROI ]
        if found:
            combined_fgmask = mog2_fgmask.copy()
            x_min, x_max, y_min, y_max = roi

            # draw ROI
            frame_contours = frame.copy()
            cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)
            cv2.imshow("frame ROI", frame_contours)

            mog2_fgmask_copy = mog2_fgmask.copy()
            cv2.rectangle(mog2_fgmask_copy, (x_min, y_min), (x_max, y_max), 127, 2)
            cv2.imshow("mog2 ROI", mog2_fgmask_copy)

            tmb_fgmask_copy = tmb_fgmask.copy()
            cv2.rectangle(tmb_fgmask_copy, (x_min, y_min), (x_max, y_max), 127, 2)
            cv2.imshow("tmb ROI", tmb_fgmask_copy)

            combined_fgmask[y_min:y_max, x_min:x_max] = cv2.bitwise_or(mog2_fgmask[y_min:y_max, x_min:x_max], tmb_fgmask[y_min:y_max, x_min:x_max])    
            combined_fgmask = cv2.morphologyEx(combined_fgmask, cv2.MORPH_CLOSE, kernel, iterations=2)
            # cv2.imshow("combined_fgmask", combined_fgmask)

            # [ OVERRIDE FROZEN VALUES ]
            if fgmask_diff is not None:
                complete = cv2.bitwise_or(combined_fgmask, fgmask_diff)

                complete = cv2.dilate(complete, kernel, iterations=1)
                complete = cv2.morphologyEx(complete, cv2.MORPH_CLOSE, kernel, iterations=2)
                cv2.imshow("combinedMask + diff", complete)

                # contours_fg, _ = cv2.findContours(complete, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                
                # frame_original_contours = cv2.drawContours(frame_original, contours_fg, -1, (0,0,255), cv2.FILLED)
                # cv2.imshow("frame", frame_original_contours)

                output = cv2.connectedComponentsWithStats(complete, connectivity=8)
                (numLabels, labels, stats, centroids) = output
                # labels is a mask with an id for each pixel
                filtered = labels.copy()
                for i in range(0, numLabels):
                    if stats[i, cv2.CC_STAT_AREA] < 200:
                        filtered[filtered == i] = 0
                
                labeled = imshow_components(filtered)
                cv2.imshow("labeled", labeled)
                # labeled = cv2.cvtColor(labeled, cv2.COLOR_BGR2GRAY)

                # contours_fg, _ = cv2.findContours(labeled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                # frame_original_contours = cv2.drawContours(frame_original, contours_fg, -1, (0,0,255), cv2.FILLED)
                # cv2.imshow("frame", frame_original_contours)


            # [ STATIC OBJECTS ]
            fgmask_diff_new = cv2.bitwise_and(tmb_fgmask, cv2.bitwise_not(combined_fgmask))
            if fgmask_diff is None:
                fgmask_diff = fgmask_diff_new
            
            fgmask_diff = cv2.bitwise_or(fgmask_diff, fgmask_diff_new)
            #fgmask_diff = cv2.medianBlur(fgmask_diff, 3)
            #fgmask_diff = cv2.dilate(fgmask_diff, None, iterations=3)
            cv2.imshow("diff (= BOOKS) = (median - combined)", fgmask_diff)
        
        


    cmd = cv2.waitKey(0)    
    if cmd == ord("q"):
        break
    if cmd == ord("n"):
        continue

end = time.time()

print(f"[I] Elapsed time: {end - start} s")

cap.release()
cv2.destroyAllWindows()
